### Paso 1: Importar librerias y funciones 

In [17]:
import pandas as pd
import os
import numpy as np
import sqlite3

from utils.data_processing import (
    cargar_y_limpiar_vuelos,
    cargar_y_limpiar_feriados,
    unir_datasets,
    normalizar_provincias
)

### --- Configuración de Rutas ---

In [3]:

# Los archivos de origen están en la carpeta 'bronce' (un nivel arriba)
path_vuelos_bronce = 'data/bronze/20250825/base_microdatos.csv'
path_feriados_bronce = 'data/bronze/20250825/FeriadosLargos.xlsx'

# La ruta de salida para la capa 'plata'
path_plata = 'data/silver' # El punto indica que se guardará en la misma carpeta del cuaderno
output_filename = 'vuelos_unificados.csv'


In [16]:
# Cargar el dataset
df_vuelos_original = pd.read_csv(path_vuelos_bronce)
print(df_vuelos_original.columns)

Index(['indice_tiempo', 'clasificacion_vuelo', 'clase_vuelo', 'aerolinea',
       'origen_oaci', 'origen_aeropuerto', 'origen_localidad',
       'origen_provincia', 'origen_pais', 'origen_continente', 'destino_oaci',
       'destino_aeropuerto', 'destino_localidad', 'destino_provincia',
       'destino_pais', 'destino_continente', 'pasajeros', 'asientos',
       'vuelos'],
      dtype='object')


In [5]:
# Renombrar la columna de tiempo para mayor claridad
df_vuelos_original.rename(columns={'indice_tiempo': 'fecha_vuelos'}, inplace=True)
print(df_vuelos_original.head())

  fecha_vuelos clasificacion_vuelo clase_vuelo              aerolinea  \
0   2017-01-01            Cabotaje  No Regular    Andes Líneas Aéreas   
1   2017-01-01            Cabotaje  No Regular    Andes Líneas Aéreas   
2   2017-01-01            Cabotaje  No Regular    Andes Líneas Aéreas   
3   2017-01-01            Cabotaje     Regular  Aerolíneas Argentinas   
4   2017-01-01            Cabotaje     Regular  Aerolíneas Argentinas   

  origen_oaci                          origen_aeropuerto  \
0        SAAV                  Aeropuerto de Sauce Viejo   
1        SASA    Aeropuerto Int. Martín Miguel de Güemes   
2        SAZS       Aeropuerto Int. Tte. Luis Candelaria   
3        SAAR  Aeropuerto Int. de Rosario Islas Malvinas   
4        SAAR  Aeropuerto Int. de Rosario Islas Malvinas   

          origen_localidad origen_provincia origen_pais origen_continente  \
0                 Santa Fe         Santa Fe   Argentina   América del Sur   
1                    Salta            Salta   

In [6]:
# Seleccionar solo las columnas que necesitas para el análisis
columnas_a_mantener = [
    'fecha_vuelos',
    'origen_provincia',
    'destino_provincia',
    'origen_pais',
    'pasajeros',
    'asientos',
    'vuelos'
]

df_vuelos = df_vuelos_original[columnas_a_mantener].copy()
df_vuelos.tail()

,fecha_vuelos,origen_provincia,destino_provincia,origen_pais,pasajeros,asientos,vuelos
929484,2025-06-30,Mendoza,NaN,Argentina,172,186,1
929485,2025-06-30,NaN,Ciudad Autónoma de Buenos Aires,Chile,441,558,3
929486,2025-06-30,NaN,Mendoza,Chile,135,186,1
929487,2025-06-30,NaN,Ciudad Autónoma de Buenos Aires,Perú,159,186,1
929488,2025-06-30,NaN,Buenos Aires,Estados Unidos,269,269,1


In [7]:
# Convertir a formato de fecha
# Usa '%Y-%m-%d' ya que tus datos tienen este formato
df_vuelos['fecha_vuelos'] = pd.to_datetime(
    df_vuelos['fecha_vuelos'],
    format='%Y-%m-%d',
    errors='coerce'
)

In [8]:
filtro_2024 = df_vuelos['fecha_vuelos'].dt.year
df_vuelos = df_vuelos[filtro_2024 == 2024]
df_vuelos

,fecha_vuelos,origen_provincia,destino_provincia,origen_pais,pasajeros,asientos,vuelos
739933,2024-01-01,Córdoba,Río Negro,Argentina,181,189,1
739934,2024-01-01,"Tierra del Fuego, Antártida e Islas del Atlánt...",Buenos Aires,Argentina,72,85,1
739935,2024-01-01,Córdoba,Río Negro,Argentina,181,189,1
739936,2024-01-01,Río Negro,Córdoba,Argentina,275,284,2
739937,2024-01-01,Río Negro,Buenos Aires,Argentina,91,95,1
...,...,...,...,...,...,...,...
865426,2024-12-31,NaN,Buenos Aires,Perú,134,186,1
865427,2024-12-31,Buenos Aires,NaN,Argentina,288,329,1
865428,2024-12-31,NaN,Buenos Aires,Brasil,102,329,1
865429,2024-12-31,NaN,Buenos Aires,Estados Unidos,335,536,2


In [9]:
# Manejar valores nulos en columnas de provincia
    # Los valores nulos en estas columnas indican que el vuelo es internacional
df_vuelos['origen_provincia'] = df_vuelos['origen_provincia'].fillna('Internacional')
df_vuelos['destino_provincia'] = df_vuelos['destino_provincia'].fillna('Internacional')

    # Manejar nulos en columnas numéricas y asegurarse de que sean de tipo entero
for col in ['pasajeros', 'asientos', 'vuelos']:
        df_vuelos[col] = df_vuelos[col].fillna(0).astype(int)

    # Calcular la tasa de ocupación, evitando la división por cero
    # Se reemplaza np.inf por 0
df_vuelos['ocupacion'] = np.where(
df_vuelos['asientos'] > 0,
(df_vuelos['pasajeros'] / df_vuelos['asientos']) * 100,
    0
)

# Redondea la columna 'ocupacion' a 2 decimales
df_vuelos['ocupacion'] = df_vuelos['ocupacion'].round(2)

In [10]:
df_vuelos.head()

,fecha_vuelos,origen_provincia,destino_provincia,origen_pais,pasajeros,asientos,vuelos,ocupacion
739933,2024-01-01,Córdoba,Río Negro,Argentina,181,189,1,95.77
739934,2024-01-01,"Tierra del Fuego, Antártida e Islas del Atlánt...",Buenos Aires,Argentina,72,85,1,84.71
739935,2024-01-01,Córdoba,Río Negro,Argentina,181,189,1,95.77
739936,2024-01-01,Río Negro,Córdoba,Argentina,275,284,2,96.83
739937,2024-01-01,Río Negro,Buenos Aires,Argentina,91,95,1,95.79


### --- Paso 1: Cargar y limpiar los datasets usando las funciones ---

In [11]:
df_feriados = pd.read_excel(path_feriados_bronce)
 # Renombrar la columna 'indice_tiempo' a 'fecha_fin_largo'
df_feriados.rename(columns={'indice_tiempo': 'fecha_fin_largo'}, inplace=True)

df_feriados['fecha_fin_largo'] = pd.to_datetime(df_feriados[ 'fecha_fin_largo'])

In [12]:

# print("Cargando y limpiando el dataset de vuelos desde la capa bronce...")
# df_vuelos_limpio = cargar_y_limpiar_vuelos(path_vuelos_bronce)
# df_vuelos_limpio.shape
# # print(f"Dataset de vuelos limpio. Filas: {df_vuelos_limpio.head()}")

print("\nCargando y limpiando el dataset de feriados...")
df_feriados_limpio = cargar_y_limpiar_feriados(path_feriados_bronce)
print(f"Dataset de feriados limpio. Filas: {len(df_feriados_limpio)}")




Cargando y limpiando el dataset de feriados...
Dataset de feriados limpio. Filas: 23


### --- Paso 2: Unir los datasets ---

In [13]:


print("\nUniendo los datasets para identificar los días de feriado largo...")
df_unificado = unir_datasets(df_vuelos, df_feriados_limpio)
print("Datasets unidos exitosamente.")
print(df_unificado.head())



Uniendo los datasets para identificar los días de feriado largo...
Datasets unidos exitosamente.
       fecha_vuelos                                   origen_provincia  \
739933   2024-01-01                                            Córdoba   
739934   2024-01-01  Tierra del Fuego, Antártida e Islas del Atlánt...   
739935   2024-01-01                                            Córdoba   
739936   2024-01-01                                          Río Negro   
739937   2024-01-01                                          Río Negro   

       destino_provincia origen_pais  pasajeros  asientos  vuelos  ocupacion  \
739933         Río Negro   Argentina        181       189       1      95.77   
739934      Buenos Aires   Argentina         72        85       1      84.71   
739935         Río Negro   Argentina        181       189       1      95.77   
739936           Córdoba   Argentina        275       284       2      96.83   
739937      Buenos Aires   Argentina         91        95

### --- Paso 3: Guardar el dataset unificado en la capa plata ---

In [14]:


ruta_salida_plata = os.path.join(path_plata, output_filename)
print(f"\nGuardando el dataset unificado en la capa plata en: {ruta_salida_plata}")
df_unificado.to_csv(ruta_salida_plata, index=False)

print("Proceso completado. El archivo de la capa plata está listo para el análisis en la capa oro.")


Guardando el dataset unificado en la capa plata en: data/silver\vuelos_unificados.csv
Proceso completado. El archivo de la capa plata está listo para el análisis en la capa oro.


In [21]:
# Paso 3: Normalizar los datos
df_vuelos_normalizado, df_provincias = normalizar_provincias(df_unificado)
print("Datos normalizados. Ahora se tienen dos DataFrames.")
print(df_vuelos_normalizado.head())
print(df_provincias)

Datos normalizados. Ahora se tienen dos DataFrames.
       fecha_vuelos origen_pais  pasajeros  asientos  vuelos  ocupacion  \
739933   2024-01-01   Argentina        181       189       1      95.77   
739934   2024-01-01   Argentina         72        85       1      84.71   
739935   2024-01-01   Argentina        181       189       1      95.77   
739936   2024-01-01   Argentina        275       284       2      96.83   
739937   2024-01-01   Argentina         91        95       1      95.79   

        es_feriado_largo  id_origen_provincia  id_destino_provincia  
739933              True                    1                     3  
739934              True                    2                     7  
739935              True                    1                     3  
739936              True                    3                     1  
739937              True                    3                     7  
                                            provincia  id_provincia
0        

In [23]:
df_vuelos_normalizado.columns

Index(['fecha_vuelos', 'origen_pais', 'pasajeros', 'asientos', 'vuelos',
       'ocupacion', 'es_feriado_largo', 'id_origen_provincia',
       'id_destino_provincia'],
      dtype='object')

In [25]:
# Paso 4: Cargar ambas tablas en la base de datos
conexion = sqlite3.connect('analisis_vuelos.db')

# Cargar el DataFrame de vuelos normalizado en su tabla
df_vuelos_normalizado.to_sql('vuelos_normalizados', conexion, if_exists='replace', index=False)

# Cargar el DataFrame de provincias en su tabla
df_provincias.to_sql('provincias', conexion, if_exists='replace', index=False)


print("Las tablas 'vuelos_normalizados' y 'provincias' han sido creadas en la base de datos.")

Las tablas 'vuelos_normalizados' y 'provincias' han sido creadas en la base de datos.
